In [ ]:
import os
import pandas
import time
os.chdir('/home/idies/workspace/Temporary/raddick/cra_scratch')
os.getcwd()

In [ ]:
s = time.time()
loans_df = pandas.read_csv('loans_df.csv', encoding='utf-8')
loans_df = loans_df.set_index('rownumber')
loans_df_bk = loans_df
e = time.time()
print('Loaded in {0:,.0f} seconds.'.format(e-s))

In [ ]:
# loans_df.sample(3).T
loans_df = loans_df_bk
print('Loaded from backup.')

respondents_df = pandas.read_csv('/home/idies/workspace/Storage/raddick/raddick_cra/respondentid.csv', encoding='utf-8', index_col='respondentID')
print('Loaded respondentIDs (n = {0:,.0f}).'.format(len(respondents_df['institution_name'].drop_duplicates())))
print('\nAdding institution (from respondentID)...')
loans_df = loans_df.join(respondents_df, how='left', on='respondentID')
# There are only 87 unique institutions represented?
print('Found institutions for {0:,.0f} ({1:,.0f} unique).'.format(len(loans_df['institution_name'].notnull()), len(loans_df['institution_name'].drop_duplicates().notnull())))
loans_df['institution_name'] = loans_df['institution_name'].fillna('Unknown')


print('Converting assessment_area_number to numeric...')
loans_df.loc[loans_df['assessment_area_number'] == 'A  ', 'assessment_area_number'] = '-1'
loans_df['assessment_area_number'] = pandas.to_numeric(loans_df['assessment_area_number'], errors='coerce')


#loans_df.sample(3)
print('\nAssigning codes...')
print('agency_code -> agency...')
loans_df.assign(agency='')
loans_df.loc[loans_df['agency_code'] == 1, 'agency'] = 'OCC'
loans_df.loc[loans_df['agency_code'] == 2, 'agency'] = 'FRS'
loans_df.loc[loans_df['agency_code'] == 3, 'agency'] = 'FDIC'
loans_df.loc[loans_df['agency_code'] == 4, 'agency'] = 'OTS'

print('loan_type -> loan_to...')
loans_df.assign(loan_to='')
loans_df.loc[loans_df['loan_type'] == 4, 'loan_to'] = 'Business'
loans_df.loc[loans_df['loan_type'] == 5, 'loan_to'] = 'Farm'

print('action_taken_type -> action_taken...')
loans_df.assign(action_taken='')
loans_df.loc[loans_df['action_taken_type'] == 1, 'action_taken'] = 'Origination'
loans_df.loc[loans_df['action_taken_type'] == 6, 'action_taken'] = 'Purchase'

#print('state -> state_name')
#statecodes_df = pandas.read_csv(
#    '/home/idies/workspace/Storage/raddick/persistent/cra/metadata/statecodes.csv', 
#    encoding='utf-8', index_col='STATE')
#statecodes_df = statecodes_df.rename(columns={'STUSAB': 'state_name'})
#statecodes_df.index.name = 'state'
#loans_df = loans_df.join(statecodes_df, how='left', on='state')

print('income_group_total -> income_group')
loans_df['income_group_total'] = pandas.to_numeric(loans_df['income_group_total'], errors='coerce')

loans_df.assign(income_group='')
loans_df.loc[loans_df['income_group_total'] == 1, 'income_group'] = '< 10% of Median Family Income (MFI)'
loans_df.loc[loans_df['income_group_total'] == 2, 'income_group'] = '10% to 20% of MFI'
loans_df.loc[loans_df['income_group_total'] == 3, 'income_group'] = '20% to 30% of MFI'
loans_df.loc[loans_df['income_group_total'] == 4, 'income_group'] = '30% to 40% of MFI'
loans_df.loc[loans_df['income_group_total'] == 5, 'income_group'] = '40% to 50% of MFI'
loans_df.loc[loans_df['income_group_total'] == 6, 'income_group'] = '50% to 60% of MFI'
loans_df.loc[loans_df['income_group_total'] == 7, 'income_group'] = '60% to 70% of MFI'
loans_df.loc[loans_df['income_group_total'] == 8, 'income_group'] = '70% to 80% of MFI'
loans_df.loc[loans_df['income_group_total'] == 9, 'income_group'] = '80% to 90% of MFI'
loans_df.loc[loans_df['income_group_total'] == 10, 'income_group'] = '90% to 100% of MFI'
loans_df.loc[loans_df['income_group_total'] == 11, 'income_group'] = '100% to 110% of MFI'
loans_df.loc[loans_df['income_group_total'] == 12, 'income_group'] = '110% to 120% of MFI'
loans_df.loc[loans_df['income_group_total'] == 13, 'income_group'] = '> 120% of MFI'
loans_df.loc[loans_df['income_group_total'] == 14, 'income_group'] = 'MFI not known (income percentage = 0)'
loans_df.loc[loans_df['income_group_total'] == 15, 'income_group'] = 'Tract not Known (reported as NA)'
loans_df.loc[loans_df['income_group_total'] == 101, 'income_group'] = 'Low Income (< 50% of MFI) - excluding 0)'
loans_df.loc[loans_df['income_group_total'] == 102, 'income_group'] = 'Moderate Income (50% to 80% of MFI)'
loans_df.loc[loans_df['income_group_total'] == 103, 'income_group'] = 'Middle Income (80% to 120% of MFI)'
loans_df.loc[loans_df['income_group_total'] == 104, 'income_group'] = 'Upper Income (> 120% of MFI)'
loans_df.loc[loans_df['income_group_total'] == 105, 'income_group'] = 'Income Not Known (0)'
loans_df.loc[loans_df['income_group_total'] == 106, 'income_group'] = 'Tract not Known (NA)'

print('\nMultiplying amount columns by 1000 to convert to real dollars...')
amtloancolumns = ['amtLoans1', 'amtLoans100k', 'amtLoans250k', 'amtLoansToSmallest', 'amtLoansAff']
for thiscol in amtloancolumns:
    print(thiscol+'...')
    loans_df[thiscol] = 1000*loans_df[thiscol]#.apply(lambda x: 1000*x)

# Read inflation data, which comes from: https://data.bls.gov/pdq/SurveyOutputServlet
print('\nLoading inflation data...')
inflation_df = pandas.read_csv('/home/idies/workspace/Storage/raddick/persistent/cra/metadata/inflation_to_2016.csv', encoding='utf-8')
inflation_df = inflation_df[(inflation_df['Year'] >= 2004) & (inflation_df['Year'] <= 2016)]
inflation_df = inflation_df.drop(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov'], axis=1)
inflation_df = inflation_df.rename(columns={'Year': 'activity_year', 'Dec': 'inflation_factor'})
inflation_df = inflation_df.set_index('activity_year')

loans_df = loans_df.join(inflation_df, how='left', on='activity_year')

# adjust
print('Adjusting loan amount columns...')

print('amtLoans1 -> amtLoans1_adjusted...')
loans_df = loans_df.assign(amtLoans1_adjusted = loans_df['amtLoans1'] * loans_df['inflation_factor'])
print('amtLoans100k -> amtLoans100k_adjusted...')
loans_df = loans_df.assign(amtLoans100k_adjusted = loans_df['amtLoans100k'] * loans_df['inflation_factor'])
print('amtLoans250k -> amtLoans250k_adjusted...')
loans_df = loans_df.assign(amtLoans250k_adjusted = loans_df['amtLoans250k'] * loans_df['inflation_factor'])
print('amtLoansToSmallest -> amtLoansToSmallest_adjusted...')
loans_df = loans_df.assign(amtLoansToSmallest_adjusted = loans_df['amtLoansToSmallest'] * loans_df['inflation_factor'])
print('amtLoansAff -> amtLoansAff_adjusted...')
loans_df = loans_df.assign(amtLoansAff_adjusted = loans_df['amtLoansAff'] * loans_df['inflation_factor'])

# remove tableID, it is redundant with loan_type and action_taken_type
loans_df = loans_df.drop('tableID', axis=1)

print('\nWriting outfile...')
loans_df.to_csv('loans_processed.csv', encoding='utf-8')
print('Outfile written')

loans_df_bk = loans_df

print('\nDONE!')


# Get MSA names (this gives memory error, figure out later)

In [ ]:
print('Loading from backup...')
loans_df = loans_df_bk
print('Converting MSA to numeric...')
loans_df['msa'] = pandas.to_numeric(loans_df['msa'], errors='coerce')

print('Reading MSA list...')
msa_df = pandas.read_csv('/home/idies/workspace/Storage/raddick/persistent/cra/metadata/msacodes.csv', encoding='utf-8')

print('Merging...')
s = time.time()
msa_df = msa_df.rename(columns={'msa_code': 'msa'})

#loans_df = loans_df.merge(
#    msa_df[['msa_code', 'msa_title']][
#        msa_df['msa_code'].notnull()
#    ], how='left', left_on='msa', right_on='msa_code')

loans_df = loans_df.merge(msa_df[['msa', 'msa_title']], how='left', on='msa')
e = time.time()
print('Merged in {0:,.0f} seconds.'.format(e-s))

loans_df.groupby('msa_title').size().sort_values(ascending=False)

In [ ]:
loans_df.columns
#loans_df.groupby('report_level').size()

In [ ]:
#loans_df.columns
#loans_df[loans_df['state'] == 24].groupby('msa').size()#.sort_values()

loans_df = loans_df_bk

#os.getcwd()
s = time.time()
msa_df = pandas.read_csv('/home/idies/workspace/Storage/raddick/persistent/cra/metadata/msacodes.csv', encoding='utf-8')
e = time.time()
print('Loaded {0:,.0f} metro areas in {1:,.0f} seconds.'.format(len(msa_df), e-s))

#msa_df.groupby('cbsa_code').size().sort_values(ascending=False)
#msa_df['cbsa_code'].drop_duplicates()
#msa_df
#loans_df.groupby('msa').size()
print('\nConverting to numeric...')
s = time.time()
loans_df['msa'] = pandas.to_numeric(loans_df['msa'], downcast='integer', errors='coerce')
e = time.time()
print('Done in {0:,.0f} seconds'.format(e-s))

print('\nMerging...')
s = time.time()
print('By CBSA...')
loans_df = loans_df.merge(
    msa_df[['cbsa_code', 'cbsa_title']], 
    how='left', left_on='msa', right_on='cbsa_code')

print('By MSA...')
loans_df = loans_df[(loans_df['cbsa_title'].isnull()) & (loans_df['msa'].notnull())].merge(msa_df[['msa_code', 'msa_title']], how='left', left_on='msa', right_on='msa_code')

e = time.time()
print('Done in {0:,.0f} seconds.'.format(e-s))
loans_df.gropuby('msa_title').size().sort_values(ascending=False)
#loans_df.groupby('msa').size()
#msa_df.dtypes
#loans_df.groupby('cbsa_title').size()

#loans_df = loans_df.merge(
#    msa_df[['cbsa_code', 'cbsa_title']], 
#    how='left', left_on='msa', right_on='cbsa_code')
